## create modeled tables for google mobility dataset

In [1]:
!bq --location=US mk --dataset google_modeled

Dataset 'spry-cosine-266801:google_modeled' successfully created.


In [2]:
%%bigquery
create or replace table google_modeled.mobility_report as
select null as id, *
from google_staging.mobility_report

""


create farm_fingerprint id

In [3]:
%%bigquery
update google_modeled.mobility_report set id = FARM_FINGERPRINT(country_region)
where sub_region_1 is null and sub_region_2 is null

""


In [4]:
%%bigquery
update google_modeled.mobility_report set id = FARM_FINGERPRINT(concat(sub_region_2,country_region))
where sub_region_1 is null and sub_region_2 is not null

""


In [5]:
%%bigquery
update google_modeled.mobility_report set id = FARM_FINGERPRINT(concat(sub_region_1,country_region))
where sub_region_1 is not null and sub_region_2 is null

""


In [6]:
%%bigquery
update google_modeled.mobility_report set id = FARM_FINGERPRINT(concat(sub_region_1, sub_region_2, country_region))
where sub_region_1 is not null and sub_region_2 is not null

""


In [7]:
#check for null

In [8]:
%%bigquery
select count(*) as null_id_count
from google_modeled.mobility_report
where id is null

,null_id_count
0,0


In [9]:
%%bigquery
select id, sub_region_1, sub_region_2, country_region
from google_modeled.mobility_report
order by sub_region_1, sub_region_2, country_region
limit 5

,id,sub_region_1,sub_region_2,country_region
0,8778414404485170876,None,None,Afghanistan
1,8778414404485170876,None,None,Afghanistan
2,8778414404485170876,None,None,Afghanistan
3,8778414404485170876,None,None,Afghanistan
4,8778414404485170876,None,None,Afghanistan


create location table

In [10]:
%%bigquery
create or replace table google_modeled.location as
select distinct id, country_region_code, sub_region_1, sub_region_2, country_region
from google_modeled.mobility_report

""


In [1]:
%%bigquery
select * from google_modeled.location
limit 12

,id,country_region_code,sub_region_1,sub_region_2,country_region
0,3303823304613400106,PE,Amazonas,None,Peru
1,6430205748557464685,PE,Metropolitan Municipality of Lima,None,Peru
2,2064989517363615664,PE,Ica,None,Peru
3,978267926391153175,PE,Puno,None,Peru
4,1291297243406899901,PE,Arequipa,None,Peru
5,-2162946003039821225,PE,Huanuco,None,Peru
6,-1646237429549995862,PE,Ucayali,None,Peru
7,-5672759309895552356,PE,Huancavelica,None,Peru
8,-1778956403537053803,PE,Junin,None,Peru
9,4211496354259836213,PE,Pasco,None,Peru


In [11]:
%%bigquery
select count(*) as location_count from
google_modeled.location

,location_count
0,4166


create mobility table

In [12]:
%%bigquery
create or replace table google_modeled.mobility_event as
select id as location_id, date, retail_and_recreation_percent_change_from_baseline as retail_and_rec, grocery_and_pharmacy_percent_change_from_baseline as 
grocery_and_pharm, parks_percent_change_from_baseline as parks, transit_stations_percent_change_from_baseline as transit_stations, 
workplaces_percent_change_from_baseline as workplaces, residential_percent_change_from_baseline as residential
from google_modeled.mobility_report

""


In [13]:
%%bigquery
select * from google_modeled.mobility_event
order by date
limit 5

,location_id,date,retail_and_rec,grocery_and_pharm,parks,transit_stations,workplaces,residential
0,-9132527727246512384,2020-02-15,5,1,None,None,0,None
1,4938801341273031425,2020-02-15,13,2,None,None,-3,None
2,8586119336968730113,2020-02-15,-3,2,None,None,5,None
3,-5863261855060669952,2020-02-15,3,1,None,None,3,None
4,-3682165675596349440,2020-02-15,6,-1,None,None,4,None


In [14]:
%%bigquery
select * from google_modeled.mobility_event
order by date desc
limit 5

,location_id,date,retail_and_rec,grocery_and_pharm,parks,transit_stations,workplaces,residential
0,-3682165675596349440,2020-04-26,NaN,NaN,NaN,-36.0,-28,None
1,-9132527727246512384,2020-04-26,NaN,NaN,NaN,NaN,-38,None
2,99867858386480128,2020-04-26,-18.0,0.0,NaN,NaN,-27,None
3,-5863261855060669952,2020-04-26,-14.0,0.0,NaN,NaN,-28,None
4,2440160576632688640,2020-04-26,-47.0,-10.0,99.0,-62.0,-16,None
